In [2]:
# this file aims at using transfer learning, to get the result in somewhat middle part of the computational
# graph, this time I use inception_resnet_v2
import tensorflow as tf
import numpy as np
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
from inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
from data_preprocessing import image_preprocessing
slim = tf.contrib.slim

In [3]:
checkpoint_file = './inception_resnet_v2_2016_08_30.ckpt'
label_file = 'training/label.idl'
train_file = 'training/'

In [4]:
# build the computational graph from inception_resnet_v2 and extract a mid point.
images = tf.placeholder(tf.float32, shape=(None, 360, 640, 3))
images_norm = tf.divide(images, 255)
with slim.arg_scope(inception_resnet_v2_arg_scope()):
    _, end_points = inception_resnet_v2(images_norm, num_classes = 1001, is_training = True)    

Mixed_5b = end_points['Mixed_5b']
    
# define the variables that I don't need to restore
exclude = ['InceptionResnetV2/AuxLogits', 'InceptionResnetV2/Logits']
variables_to_restore = slim.get_variables_to_restore(exclude = exclude)
saver = tf.train.Saver(variables_to_restore)

In [5]:
image_preprocessor = image_preprocessing(train_file, label_file)
image_preprocessor.get_training_data()

In [11]:
with tf.Session() as sess:
    saver.restore(sess, checkpoint_file)
    result = sess.run(Mixed_5b, feed_dict = {images: image_preprocessor.next_batch()[0]})

INFO:tensorflow:Restoring parameters from ./inception_resnet_v2_2016_08_30.ckpt
